In [1]:
import os
# Set environment variables for memory management
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

In [3]:
train_dataset = load_dataset("gsm8k", "main", split="train")
test_dataset = load_dataset("gsm8k", "main", split="test")

In [4]:
test_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

In [5]:
test_dataset[0]['question'], test_dataset[0]['answer']

("Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18')

# Prompt formatting
Gemma models do not support separate 'system prompt'. See more info about the formatting here: https://ai.google.dev/gemma/docs/core/prompt-structure

In [6]:
instruction_part = "<bos><start_of_turn>user\n"
response_part = "<start_of_turn>model\n"

In [7]:
def format_example(example):
    """Format examples for Gemma model with separate question and answer fields"""
    question = example['question']
    answer = example['answer']
    # Gemma adds <bos> at the beginning of the input when generating
    # the response, so we do not need to add it to the formatted input
    # Gemma also does not have a system instruction, so we will prepend the instruction to the input
    formatted_input = f"<start_of_turn>user\nLet's solve this step by step and output the final answer after \"####\".\n{question}<end_of_turn>\n<start_of_turn>model\n"
    formatted_output = f"{answer}<end_of_turn>"
    
    return {
        "question": formatted_input,
        "answer": formatted_output,
        "original_question": question,
        "original_answer": answer
    }

In [8]:
format_example(train_dataset[0])

{'question': '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<end_of_turn>\n<start_of_turn>model\n',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72<end_of_turn>',
 'original_question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'original_answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [9]:
formatted_example = format_example(train_dataset[0])
formatted_example['question'], formatted_example['answer']

('<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<end_of_turn>\n<start_of_turn>model\n',
 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72<end_of_turn>')

In [10]:
print(f"Dataset type: {type(test_dataset)}")

Dataset type: <class 'datasets.arrow_dataset.Dataset'>


In [11]:
def format_dataset(dataset):
    formatted_dataset = dataset.map(
        format_example, 
        num_proc=4,
        desc="Formatting dataset for Gemma model"
    )
    return formatted_dataset

test_dataset_formatted = format_dataset(test_dataset)

In [12]:
test_dataset_formatted[0]

{'question': '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers\' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers\' market?<end_of_turn>\n<start_of_turn>model\n',
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18<end_of_turn>',
 'original_question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'original_answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=1

In [23]:
model_id = "google/gemma-3-1b-it"   
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Inference
Let's test our model on a few examples

In [14]:
device = model.device
inputs = tokenizer(test_dataset_formatted[12]['question'], return_tensors="pt", padding=True).to(device)
output = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    top_k=64,
    top_p=0.95,
    temperature=1.0
    )
output_text = tokenizer.decode(output[0], skip_special_tokens=False)
output_text

'<bos><start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nCarlos is planting a lemon tree. The tree will cost $90 to plant. Each year it will grow 7 lemons, which he can sell for $1.5 each. It costs $3 a year to water and feed the tree. How many years will it take before he starts earning money on the lemon tree?<end_of_turn>\n<start_of_turn>model\nLet $n$ be the number of years it takes for Carlos to start earning money on the lemon tree.\nThe cost to plant the tree is $90.\nThe tree grows 7 lemons per year, and he sells each lemon for $1.5.\nSo, the total revenue per year is $7 \\times 1.5 = 10.5$ dollars.\nThe cost to water and feed the tree is $3 per year.\nSo, the total expenses per year are $3 + 90 = 93$ dollars.\nThe profit per year is $10.5 - 93 = -82.5$ dollars.\nThis means that after 1 year, Carlos will have a loss of $82.5$ dollars.\nAfter 2 years, Carlos will have a loss of $2 \\times (-82.5) = -165$ dollars.\nAfter 3 years, Carlos

In [15]:
import re
def extract_final_answer(answer):
    """Extract the final answer after ####"""
    if "####" in answer:
        try:
            return float((answer.split("####")[-1].split(" ")[0].strip()).replace(',', ''))
        except (IndexError, ValueError):
            pass
    
    # Fallback to number extraction
    numbers = re.findall(r'-?\d*\.?\d+', answer.replace(',', ''))
    try:
        return float(numbers[-1]) if numbers else None
    except (IndexError, ValueError):
        return None


In [31]:
import re
def extract_final_answer_str(answer):
    """Extract the final answer after ####"""
    if "####" in answer:
        try:
            return (answer.split("####")[-1].split(" ")[0].strip()).replace(',', '')
        except (IndexError, ValueError):
            pass
    
    # Fallback to number extraction
    numbers = re.findall(r'-?\d*\.?\d+', answer.replace(',', ''))
    try:
        return float(numbers[-1]) if numbers else None
    except (IndexError, ValueError):
        return None


In [16]:
extract_final_answer(output_text)

-330.0

In [18]:

def evaluate_model(model, tokenizer, dataset, num_examples=100):
    correct = 0
    total = 0
    
    for i in range(min(num_examples, len(dataset))):
        # Get input and true answer
        inputs = tokenizer(dataset[i]['question'], return_tensors="pt", padding=True).to("cuda")
        true_answer = extract_final_answer(dataset[i]['original_answer'])
        
        # Generate prediction
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False,  # Deterministic for evaluation
                temperature=1.0
            )
        
        # Decode and extract prediction
        pred_text = tokenizer.decode(output[0], skip_special_tokens=False)
        pred_answer = extract_final_answer(pred_text)
        
        # Compare answers 
        if pred_answer == true_answer:
            correct += 1
        total += 1
        
        if i % 10 == 0:
            print(f"Progress: {i}/{num_examples}, Current accuracy: {correct/total:.2%}")
    
    final_accuracy = correct/total
    print(f"\nFinal accuracy: {final_accuracy:.2%}")
    return final_accuracy

In [122]:
evaluate_model(model, tokenizer, test_dataset_formatted, num_examples=100)

Progress: 0/100, Current accuracy: 0.00%
Progress: 10/100, Current accuracy: 36.36%
Progress: 20/100, Current accuracy: 38.10%
Progress: 30/100, Current accuracy: 45.16%


KeyboardInterrupt: 

In [15]:
from vllm import LLM, SamplingParams
from tqdm import tqdm


INFO 05-18 21:23:28 [__init__.py:239] Automatically detected platform cuda.


2025-05-18 21:23:29.273418: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 21:23:29.281207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747596209.290024  375040 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747596209.292730  375040 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 21:23:29.302651: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [20]:
def evaluate_with_vllm(model, dataset, batch_size=32):
            
    # Configure vLLM with memory-efficient settings
    llm = LLM(
        model=model,
        dtype=torch.bfloat16,
        trust_remote_code=True,
        gpu_memory_utilization=0.75,
        max_num_seqs=batch_size,
        tensor_parallel_size=1,
        enforce_eager=True,
    )
    
    # Use Gemma's validation sampling parameters
    sampling_params = SamplingParams(
        temperature=1,
        top_p=1.0,
        top_k=-1,
        max_tokens=512,
        
    )
        
    # Prepare evaluation loop
    correct = 0
    total = 0
    all_results = []
    
    try:
        # Convert dataset to list for proper batch processing
        test_data = list(dataset)
        #test_data = test_data[:64]
        # Process dataset in batches
        for i in tqdm(range(0, len(test_data), batch_size)):
            # Clear cache between batches
            torch.cuda.empty_cache()
            
            # Get batch
            batch_items = test_data[i:i + batch_size]
            
            # Prepare prompts
            prompts = [item['question'] for item in batch_items]
            
            print(prompts[0])

            # Generate responses using vLLM
            outputs = llm.generate(prompts, sampling_params)
            
            
            # Process each response
            for j, (output, item) in enumerate(zip(outputs, batch_items)):
                solution_str = output.outputs[0].text
                
                print('Solution: ' + solution_str[-200:])
                print('GT: ' + item['answer'][-100:])
                gt_final_answer=extract_final_answer(item['original_answer'])
                print(f"Final answer: {gt_final_answer}")

                pred_final_answer = extract_final_answer(solution_str)
                print(f"Predicted answer: {pred_final_answer}")   
                # Check if the prediction is correct
                if pred_final_answer == gt_final_answer:
                    correct += 1
                else:
                    print(f"Incorrect prediction for example {i+j}: {item['original_question']}")
                all_results.append({
                    'input': item['original_question'],
                    'predicted_answer': pred_final_answer,
                    'true_answer': gt_final_answer,
                    'solution_str': solution_str
                })
            # Update total processed examples
            total += len(batch_items)
            # Clear cache to free up memory
            torch.cuda.empty_cache()
            # Print progress
            print(f"Processed {i+j+1}/{len(test_data)} examples")
        # Final accuracy
        accuracy = correct / total if total > 0 else 0
        print(f"\nFinal accuracy: {accuracy:.2%}")   

        
    except Exception as e:
        print(f"Error during evaluation: {e}")
        if total > 0:
            print(f"Partial results available for {total} examples")
    
    finally:
        # Clean up
        torch.cuda.empty_cache()
    
    # Final results
    accuracy = correct / total if total > 0 else 0
    return {
        'accuracy': accuracy,
        'correct': correct,
        'total': total,
        'detailed_results': all_results
    }

In [25]:
evaluate_with_vllm(model_id, test_dataset_formatted, batch_size=128)

INFO 05-18 19:22:03 [config.py:717] This model supports multiple tasks: {'generate', 'reward', 'score', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 05-18 19:22:03 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 05-18 19:22:03 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 05-18 19:22:05 [__init__.py:239] Automatically detected platform cuda.


2025-05-18 19:22:05.933207: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 19:22:05.940722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747588925.949548  271259 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747588925.952268  271259 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 19:22:05.962098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO 05-18 19:22:08 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='google/gemma-3-1b-it', speculative_config=None, tokenizer='google/gemma-3-1b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=google/gemma-3-1b-it, num_scheduler_steps=1, multi_step_stream_outputs=True, enable_prefix_caching=True, chunked_prefill_enabled=True, use_async_o

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.62it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.62it/s]



INFO 05-18 19:22:10 [loader.py:458] Loading weights took 0.22 seconds
INFO 05-18 19:22:10 [gpu_model_runner.py:1347] Model loading took 1.9148 GiB and 1.231014 seconds
INFO 05-18 19:22:12 [kv_cache_utils.py:634] GPU KV cache size: 1,230,336 tokens
INFO 05-18 19:22:12 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 37.55x
INFO 05-18 19:22:12 [core.py:159] init engine (profile, create kv cache, warmup model) took 1.60 seconds
INFO 05-18 19:22:12 [core_client.py:439] Core engine process 0 ready.


...

Processed prompts:   0%|          | 0/39 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

100%|██████████| 11/11 [00:41<00:00,  3.79s/it]

Solution: a's time is 2 hours more than half Steve's time, so $T_A = \frac{1}{2} T_S + 2 = 5 + 2 = 7$ hours.
Therefore, it took Anna 7 hours to finish the difficult puzzle.

Final Answer: The final answer is 7.
GT: ore than half the time it took Steve, she took 5+2=7 hours to finish the puzzle.
#### 7<end_of_turn>
Final answer: 7.0
Predicted answer: 7.0
Solution: they eat 40 * 7 = 280 pounds of feed per week.

4. **Feed remaining after a week:** Dexter ordered 300 pounds of feed, and they eat 280 pounds, so there are 300 - 280 = 20 pounds of feed left.

####20
GT: 80>>280 pounds.
So, there are 300 - 280 = <<300-280=20>>20 pounds of feed left.
#### 20<end_of_turn>
Final answer: 20.0
Predicted answer: 20.0
Solution: ns 6 hot dogs and costs $2.
The number of packs needed is $\frac{36}{6} = 6$ packs.
The total cost is $6 \times 2 = 12$.
Therefore, John needs to spend $12.

Final Answer: The final answer is 12.
####
GT: 6/6=<<36/6=6>>6 packs of hotdogs
That means he needs to spend 6*2=$<<6*


[rank0]:[W518 19:22:54.483565857 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


{'accuracy': 0.4715693707354056,
 'correct': 622,
 'total': 1319,
 'detailed_results': [{'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
   'predicted_answer': 18.0,
   'true_answer': 18.0,
   'solution_str': "Here's how to solve the problem step-by-step:\n\n1. **Eggs remaining:** Janet starts with 16 eggs and eats 3, leaving her with 16 - 3 = 13 eggs.\n\n2. **Muffins:** She bakes muffins for friends, which use 4 eggs.\n\n3. **Eggs available for sale:** She has 13 eggs - 4 eggs = 9 eggs.\n\n4. **Daily earnings:** She sells each egg for $2, so she makes 9 * $2 = $18.\n\n####\nAnswer: $18"},
  {'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
   'predicted_answer': 3.0,
   'true_

In [17]:
def calculate_reward(pred_answer, true_answer):
    """
    Calculate reward based on the predicted and true answers.
    This is a simple example; you can modify the reward function as needed.
    """
    if pred_answer == true_answer:
        return 1.0  # Full reward for correct answer
    else:
        return 0.0  # No reward for incorrect answer

In [72]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    
    print(f"Prompts: {prompts}")
    print(f"Completions: {completions}")    
    print(f"Answer: {answer}")
    responses = completions  
    extracted_responses = [extract_final_answer(r) for r in responses]
    extracted_answers = [a for a in answer]
    
    
    # Return rewards
    return [2.0 if r == a else 0.0 
            for r, a in zip(extracted_responses, extracted_answers)]

In [ ]:
from trl import GRPOConfig, GRPOTrainer
``

INFO 05-18 21:37:36 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-18 21:37:36 [__init__.py:239] Automatically detected platform cuda.


In [75]:
training_args = GRPOConfig(
    #use_vllm = True, # use vLLM for fast inference!
    #vllm_mode="colocate",
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    logging_steps = 1,
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [20]:
train_dataset_formatted = format_dataset(train_dataset)

In [63]:
def format_example(example):
    """Format examples for Gemma model with TRL-compatible fields"""
    question = example['question']
    answer = example['answer']
    
    formatted_input = f"<start_of_turn>user\nLet's solve this step by step and output the final answer after \"####\".\n{question}<end_of_turn>\n<start_of_turn>model\n"
    formatted_output = f"{answer}<end_of_turn>"
    
    return {
        "prompt": formatted_input,  # Changed from 'question' to 'prompt'
        "completion": formatted_output,  # Changed from 'answer' to 'completion'
        "original_question": question,
        "answer": extract_final_answer(answer)
    }

# Update dataset formatting
train_dataset_formatted = format_dataset(train_dataset)


Formatting dataset for Gemma model (num_proc=4):   0%|          | 0/7473 [00:00<?, ? examples/s]

In [76]:

# Update trainer initialization
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[correctness_reward_func],
    args=training_args,
    train_dataset=train_dataset_formatted,
)

In [77]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: velic (velic-). Use `wandb login --relogin` to force relogin


Prompts: ['<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nAhmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers?<end_of_turn>\n<start_of_turn>model\n', '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nAhmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers?<end_of_turn>\n<

Step,Training Loss
1,0.000000
2,0.000000
3,0.000100
4,0.000000
5,0.000100
6,0.000100
7,0.000100
8,0.000000
9,0.000100
10,0.000100


Prompts: ['<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nThere are 15 tables in the school\'s cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken?<end_of_turn>\n<start_of_turn>model\n', '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nThere are 15 tables in the school\'s cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken?<end_of_turn>\n<start_of_turn>model\n', '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####".\nThere are 15 tables in the school\'s cafeteria. Each table can seat 10 people. Usually, only 1/10 of the seats are left unseated. How many seats are usually taken?<end_of_turn>\n<start_of_turn>model\n', '<start_of_turn>user\nLet\'s solve this step by step and output the final answer after "####"


Aborted!


KeyboardInterrupt: 